In [1]:
import pandas as pd
lines=pd.read_csv(r"English_to_Roman_Urdu changes data.csv")

In [2]:
lines.head(20)

,eng,urdu
0,how is things?,cheezein kesi hain?
1,how is life?,zindagi kesi chal rahi?
2,what is happening?,kiya horaha>
3,what is going on?,kiya chalraha?
4,hey there!,arey haan
5,what is up?,aur kiya horaha hai?
6,good mornng,subah bakhair
7,good day,acha din
8,good afternoon,dopehr bakahir
9,good evening,sham bakhair


In [3]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [4]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.urdu=lines.urdu.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.urdu=lines.urdu.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters

# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.urdu=lines.urdu.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.urdu=lines.urdu.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.urdu=lines.urdu.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.urdu = lines.urdu.apply(lambda x : 'START_ '+ x + ' _END')

lines.sample(10)

,eng,urdu
249,alright mate,START_ sab theak dost _END
226,hey there,START_ arey haan _END
69,whats going on,START_ kiya horaha hai _END
136,lovely to see you,START_ app ko dekh kar acha laga _END
222,how is things,START_ cheezein kesi hain _END
254,hi there,START_ hi haan _END
230,good afternoon,START_ dopehr bakahir _END
79,hows it going,START_ kesa chal raha hai _END
132,good to see you,START_ tumhe dekh kar acha laga _END
190,hows it going,START_ kesa chal raha hai _END


In [5]:

# Vocabulary of English
all_eng_words=set()# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print('Max Source Length:',max_length_src)

# Max Length of target sequence
lenght_list=[]
for l in lines.urdu:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print('Max Target Lenght:',max_length_tar)
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Urdu 
all_urdu_words=set()
for urdu in lines.urdu:
    for word in urdu.split():
        if word not in all_urdu_words:
            all_urdu_words.add(word)# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print('Max Source Length:',max_length_src)

# Max Length of target sequence
lenght_list=[]
for l in lines.urdu:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print('Max Target Lenght:',max_length_tar)

Max Source Length: 9
Max Target Lenght: 11
Max Source Length: 9
Max Target Lenght: 11


In [6]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_urdu_words))
num_encoder_tokens = len(all_eng_words)+1
num_decoder_tokens = len(all_urdu_words)+1
num_encoder_tokens, num_decoder_tokens

(121, 153)

In [7]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

154

In [8]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

lines = shuffle(lines)
lines.head(10)

,eng,urdu
61,i wish you all the best,START_ mai apke lye naik khuwahishaat rakhta h...
35,how is your day,START_ app ka din kesa hai _END
52,i am happy to see you again,START_ mai apko dobara dekh ke khush hun _END
93,what’s up,START_ kiya chal raha hai _END
287,have fun,START_ mazey karo _END
101,can i stay here,START_ kiya mein yahan reh sakta hun _END
288,break a leg,START_ tang khechna _END
14,how are you doing,START_ app kese hain _END
265,im glad to see you again,START_ mai app ko dobara dekh kar khush hun _END
32,hi there,START_ hi haan _END


In [9]:
# Train - Test Split
X, y = lines.eng, lines.urdu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((299,), (34,))

In [10]:
#Save the train and test dataframes for reproducing the results later, as they are shuffled.

X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [11]:
def generate_batch(X = X_train, y = y_train, batch_size = 64):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [12]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 1
epochs = 50
latent_dim = 256
print(train_samples//batch_size)
print(val_samples//batch_size)

299
34


In [13]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [14]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [16]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [17]:
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/50
299/299 [==============================] - 40s 66ms/step - loss: 3.8975 - acc: 0.2249 - val_loss: 3.2465 - val_acc: 0.2457
Epoch 2/50
299/299 [==============================] - 13s 44ms/step - loss: 3.0931 - acc: 0.3308 - val_loss: 2.6037 - val_acc: 0.4114
Epoch 3/50
299/299 [==============================] - 13s 44ms/step - loss: 2.4809 - acc: 0.4321 - val_loss: 2.1075 - val_acc: 0.5143
Epoch 4/50
299/299 [==============================] - 13s 44ms/step - loss: 1.9708 - acc: 0.5258 - val_loss: 1.8050 - val_acc: 0.5771
Epoch 5/50
299/299 [==============================] - 13s 44ms/step - loss: 1.5099 - acc: 0.6209 - val_loss: 1.4979 - val_acc: 0.6571
Epoch 6/50
299/299 [==============================] - 13s 44ms/step - loss: 1.1377 - acc: 0.7181 - val_loss: 1.2401 - val_acc: 0.7371
Epoch 7/50
299/299 [==============================] - 13s 44ms/step - loss: 0.8316 - acc: 0.8043 - val_loss: 1.0374 - val_acc: 0.7829
Epoch 8/50
299/299 [==============================] - 13s 44ms

In [18]:
#Always remember to save the weights

model.save_weights('etu_weights.h5')

In [19]:
#Load the weights, if you close the application

model.load_weights('etu_weights.h5')

In [20]:
#Inference Setup

# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [21]:
#Decode sample sequeces

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [22]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [40]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 34ms/step
Input English sentence: im glad to see you again
Actual Urdu Translation:  mai app ko dobara dekh kar khush hun 
Predicted Urdu Translation:  mai app ko dobara dekh kar khush hun 


In [41]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    y_train[k:k+1].values[0][6:-4].split()
]
print(reference)

[['mai', 'app', 'ko', 'dobara', 'dekh', 'kar', 'khush', 'hun']]


In [42]:
candidate = decoded_sentence[:-4].split()
print(candidate)
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

['mai', 'app', 'ko', 'dobara', 'dekh', 'kar', 'khush', 'hun']
BLEU score -> 1.0


In [44]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 33ms/step
Input English sentence: look who its
Actual Urdu Translation:  dekho kon hai 
Predicted Urdu Translation:  dekho kon hai 


In [45]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    y_train[k:k+1].values[0][6:-4].split()
]
print(reference)

[['dekho', 'kon', 'hai']]


In [46]:
candidate = decoded_sentence[:-4].split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

BLEU score -> 1.2213386697554703e-77


In [47]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 52ms/step
Input English sentence: happy to meet you
Actual Urdu Translation:  aap se mil ke khushi hui 
Predicted Urdu Translation:  aap se mil ke khushi hui 


In [48]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    y_train[k:k+1].values[0][6:-4].split()
]
print(reference)

[['aap', 'se', 'mil', 'ke', 'khushi', 'hui']]


In [49]:
candidate = decoded_sentence[:-4].split()
print(candidate)
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

['aap', 'se', 'mil', 'ke', 'khushi', 'hui']
BLEU score -> 1.0


In [72]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [73]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Urdu Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 47ms/step
Input English sentence: do you have some place
Actual Urdu Translation:  kiya aap ke paas koi jagah hai 
Predicted Urdu Translation:  kiya aap ke paas koi jagah hai 


In [74]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    y_test[k:k+1].values[0][6:-4].split()
]
print(reference)
candidate = decoded_sentence[:-4].split()
print(candidate)
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

[['kiya', 'aap', 'ke', 'paas', 'koi', 'jagah', 'hai']]
['kiya', 'aap', 'ke', 'paas', 'koi', 'jagah', 'hai']
BLEU score -> 1.0


In [75]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Urdu Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 49ms/step
Input English sentence: i would love to travel again
Actual Urdu Translation:  mein dobara safar krna pasand krun ga 
Predicted Urdu Translation:  mein dobara safar krna pasand krun ga 


In [76]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    y_test[k:k+1].values[0][6:-4].split()
]
print(reference)
candidate = decoded_sentence[:-4].split()
print(candidate)
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

[['mein', 'dobara', 'safar', 'krna', 'pasand', 'krun', 'ga']]
['mein', 'dobara', 'safar', 'krna', 'pasand', 'krun', 'ga']
BLEU score -> 1.0


In [66]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Urdu Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 51ms/step
Input English sentence: good to see you
Actual Urdu Translation:  tumhe dekh kar acha laga 
Predicted Urdu Translation:  tumhe dekh kar acha laga 


In [68]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    y_test[k:k+1].values[0][6:-4].split()
]
print(reference)
candidate = decoded_sentence[:-4].split()
print(candidate)
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

[['tumhe', 'dekh', 'kar', 'acha', 'laga']]
['tumhe', 'dekh', 'kar', 'acha', 'laga']
BLEU score -> 1.0
